## Práctica 2: Transformaciones geométricas
Integrantes: David García Díaz y Alejandro Bolaños García

#### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.




Este código se utiliza para aplicar transformaciones afines (rotación, escalado y traslación) a una imagen de manera interactiva. Carga una imagen y crea una ventana con trackbars que permiten ajustar los parámetros de rotación (en grados), traslación (en ambos ejes) y escalado (tanto uniformemente como por separado en los ejes X e Y). Cada vez que el usuario mueve un trackbar, el código actualiza la imagen aplicando la transformación correspondiente mediante una matriz de transformación afín.

In [2]:
import cv2 as cv
import numpy as np

# Variables globales
tx, ty = 0, 0
angle = 0
scale_x = 1.0
scale_y = 1.0
size = (1000, 1000)
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

h, w = image.shape[:2]

def update_image():
    global tx, ty, angle, scale_x, scale_y

    # Centro de la imagen para la rotación
    center = (w / 2, h / 2)

    # Crear la matriz de transformación afín
    M = cv.getRotationMatrix2D(center, angle, 1.0)
    
    # Ajustar la matriz para incluir el escalado
    M[0, 0] *= scale_x
    M[0, 1] *= scale_x
    M[1, 0] *= scale_y
    M[1, 1] *= scale_y

    # Añadir traslación
    M[0, 2] += tx
    M[1, 2] += ty

    # Aplicar la transformación afín
    transformed_image = cv.warpAffine(image, M, size)
    cv.imshow('Transformaciones', transformed_image)

# Creación de las funciiones para los valores de los trackbars
def on_trackbar_rotation(val):
    global angle
    angle = val
    update_image()

def on_trackbar_translation(val):
    global tx, ty
    tx = val - 100  
    ty = val - 100 
    update_image()

def on_trackbar_resize(val):
    global scale_x, scale_y
    scale_x = val / 100
    scale_y = val / 100
    update_image()

def on_trackbar_resize_x(val):
    global scale_x
    scale_x = val / 100
    update_image()

def on_trackbar_resize_y(val):
    global scale_y
    scale_y = val / 100
    update_image()

# Crear ventana y barras deslizantes
cv.namedWindow('Transformaciones')
cv.createTrackbar('Rotation', 'Transformaciones', 0, 360, on_trackbar_rotation)
cv.createTrackbar('Translation', 'Transformaciones', 100, 200, on_trackbar_translation)  # Centrar en 0
cv.createTrackbar('Resize', 'Transformaciones', 100, 200, on_trackbar_resize)
cv.createTrackbar('Resize X', 'Transformaciones', 100, 200, on_trackbar_resize_x)
cv.createTrackbar('Resize Y', 'Transformaciones', 100, 200, on_trackbar_resize_y)

update_image()

cv.waitKey(0)
cv.destroyAllWindows()


#### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

En este apartado se aplica una transformación de perspectiva a una imagen de manera interactiva. Carga una imagen y define los cuatro puntos de las esquinas como puntos de origen de la transformación. Luego, el usuario puede seleccionar cuatro puntos de destino en la imagen con el ratón. Una vez que se seleccionan los cuatro puntos de destino, el código calcula la matriz de transformación de perspectiva y la aplica a la imagen, creando una "imagen proyectada".

In [12]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

h, w = image.shape[:2]

# Definir los puntos de la ventana de proyección en la imagen original (cuatro esquinas)
src_points = np.float32([[0, 0], [w, 0], [w, h], [0, h]])

# Lista para almacenar los puntos de destino
dst_points = []

image_copy = image.copy()

# Función para manejar los eventos del mouse
def mouse(event, x, y, flags, param):
    global dst_points, image

    if event == cv.EVENT_LBUTTONDOWN:
        if len(dst_points) < 4:
            # Almacenar el punto seleccionado
            dst_points.append([x, y])
            cv.circle(image_copy, (x, y), 10, (0, 255, 0), -1)
            cv.imshow('Imagen original', image_copy)

        if len(dst_points) == 4:
            dst_points_np = np.float32(dst_points)

            # Calcular la matriz de transformación de perspectiva
            M = cv.getPerspectiveTransform(src_points, dst_points_np)

            # Aplicar la transformación de perspectiva
            projected_image = cv.warpPerspective(image, M, (w, h))

            cv.imshow('Imagen Proyectada', projected_image)

cv.imshow('Imagen original', image)
cv.setMouseCallback('Imagen original', mouse)

cv.waitKey(0)
cv.destroyAllWindows()


#### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

Este código aplica una distorsión de lente a una imagen la cual, primero, carga y después define una matriz de la supuesta cámara, asumiendo una matriz de identidad escalada al tamaño de la imagen. También inicializa los coeficientes de distorsión (K1 y K2) en cero. Luego, crea una ventana con trackbars que permiten al usuario ajustar estos coeficientes en un rango de -1.0 a 1.0. Cada vez que se cambia el valor de un trackbar, se actualiza la imagen con la distorsión aplicada, mostrando el efecto en tiempo real. Esto permite visualizar cómo los coeficientes de distorsión afectan a la imagen al simular diferentes distorsiones típicas de lentes.

In [4]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

h, w = image.shape[:2]

# Definir los coeficientes iniciales de distorsión
dist_coeffs = np.zeros((4,1), np.float64)

# Matriz de la cámara (suponiendo una matriz de identidad)
camera_matrix = np.array([[w, 0, w / 2],
                          [0, h, h / 2],
                          [0, 0, 1]], dtype=np.float32)

# Función para actualizar la imagen distorsionada
def update_image():
    global dist_coeffs, camera_matrix, image
    
    distorted_image = cv.undistort(image, camera_matrix, dist_coeffs)
    
    cv.imshow('Distorsion de la Lente', distorted_image)

# Funciones para los eventos de los trackbars
def on_trackbar_k1(val):
    dist_coeffs[0, 0] = (val - 1000) / 1000.0
    update_image()

def on_trackbar_k2(val):
    dist_coeffs[1, 0] = (val - 1000) / 1000.0
    update_image()

cv.namedWindow('Distorsion de la Lente')

# Crear los trackbars para ajustar los coeficientes de distorsión
cv.createTrackbar('K1', 'Distorsion de la Lente', 1000, 2000, on_trackbar_k1)
cv.createTrackbar('K2', 'Distorsion de la Lente', 1000, 2000, on_trackbar_k2)

update_image()

cv.waitKey(0)
cv.destroyAllWindows()


### Opcionales

##### - Marcar punto de giro con ratón
##### - Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real. 
##### - Todo para imagen y vídeo

Este código permite aplicar transformaciones interactivas (rotación, escalado y traslación) a una imagen o vídeo. Dependiendo de la configuración, carga una imagen o captura video de la cámara y muestra los efectos de las transformaciones mediante una matriz afín. El usuario puede ajustar la rotación y el escalado con trackbars, mientras que la traslación se controla al arrastrar con el ratón. Al hacer clic en la imagen, se puede mover el centro de rotación, y al arrastrar con doble clic, se modifica la posición en pantalla.

In [11]:
import cv2 as cv
import numpy as np

# Variables globales
tx, ty = 0, 0
angle = 0
scale_x = 1.0
scale_y = 1.0
center = None
dragging = False
prev_x, prev_y = 0, 0

# Define si quieres usar una imagen o un vídeo
USE_IMAGE = True
IMAGE_PATH = 'images/cats.png'

# Carga la imagen o el vídeo de la cámara en función de la variable USE_IMAGE
if USE_IMAGE:
    image = cv.imread(IMAGE_PATH, cv.IMREAD_COLOR)
    if image is None:
        raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")
    size = (image.shape[1], image.shape[0])
else:
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("No se puede acceder a la cámara.")
    size = (640, 480)


# Función para actualizar la imagen con las transformaciones
def update_image(frame):
    global tx, ty, angle, scale_x, scale_y, center

    h, w = frame.shape[:2]

    if center is None:
        center = (w // 2, h // 2)

    # Crear la matriz de transformación afín
    M = cv.getRotationMatrix2D(center, angle, 1.0)

    # Ajustar la matriz para incluir el escalado
    M[0, 0] *= scale_x
    M[0, 1] *= scale_x
    M[1, 0] *= scale_y
    M[1, 1] *= scale_y

    # Añadir traslación
    M[0, 2] += tx
    M[1, 2] += ty

    # Aplicar la transformación afín
    transformed_image = cv.warpAffine(frame, M, size)
    cv.imshow('Transformaciones', transformed_image)


# Funciones de manejo de los trackbars
def on_trackbar_rotation(val):
    global angle
    angle = val
    update_image(image if USE_IMAGE else frame)

def on_trackbar_resize(val):
    global scale_x, scale_y
    scale_x = val / 100
    scale_y = val / 100
    update_image(image if USE_IMAGE else frame)

def on_trackbar_resize_x(val):
    global scale_x
    scale_x = val / 100
    update_image(image if USE_IMAGE else frame)

def on_trackbar_resize_y(val):
    global scale_y
    scale_y = val / 100
    update_image(image if USE_IMAGE else frame)


# Funciones de manejo del ratón
def on_mouse(event, x, y, flags, param):
    global center, tx, ty, prev_x, prev_y, dragging
    
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
    
    elif event == cv.EVENT_MOUSEMOVE:
        if dragging:
            dx = x - prev_x
            dy = y - prev_y
            tx += dx
            ty += dy
            prev_x, prev_y = x, y
            update_image(image if USE_IMAGE else frame)
    
    elif event == cv.EVENT_LBUTTONUP:
        dragging = False
    
    elif event == cv.EVENT_LBUTTONDBLCLK:
        dragging = True
        prev_x, prev_y = x, y

# Crear ventana y trackbars
cv.namedWindow('Transformaciones')
cv.createTrackbar('Rotation', 'Transformaciones', 0, 360, on_trackbar_rotation)
cv.createTrackbar('Resize', 'Transformaciones', 100, 200, on_trackbar_resize)
cv.createTrackbar('Resize X', 'Transformaciones', 100, 200, on_trackbar_resize_x)
cv.createTrackbar('Resize Y', 'Transformaciones', 100, 200, on_trackbar_resize_y)
cv.setMouseCallback('Transformaciones', on_mouse)

# Bucle de ejecución
if USE_IMAGE:
    # Para imagen estática, solo necesitamos actualizar la imagen una vez
    update_image(image)
    cv.waitKey(0)
else:
    # Para vídeo, repetimos en cada fotograma
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        update_image(frame)
        if cv.waitKey(1) & 0xFF == 27: 
            break
    cap.release()

cv.destroyAllWindows()


#### Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en una imagen destino y realizar la transformación afín.

Este código permite seleccionar tres puntos de referencia en dos imágenes diferentes para aplicar una transformación afín que alinea la imagen de origen con la imagen de destino en función de esos puntos. Una vez que se han seleccionado tres puntos en ambas imágenes, se calcula y aplica la transformación afín, mapeando los puntos de la imagen de origen a los de la imagen de destino

In [4]:
import cv2 as cv
import numpy as np

image_src = cv.imread('images/cats.png', cv.IMREAD_COLOR)  # Imagen de origen
image_dst = cv.imread('images/cats.png', cv.IMREAD_COLOR)  # Imagen de destino

if image_src is None or image_dst is None:
    raise FileNotFoundError("Una o ambas imágenes no se pudieron cargar. Verifica las rutas.")

# Listas para almacenar los puntos seleccionados
src_points = []
dst_points = []

image_src_copy = image_src.copy()
image_dst_copy = image_dst.copy()

# Función para manejar los eventos del mouse en la imagen de origen
def mouse_src(event, x, y, flags, param):
    global src_points

    if event == cv.EVENT_LBUTTONDOWN and len(src_points) < 3:
        # Almacenar el punto seleccionado en la lista de puntos de origen
        src_points.append([x, y])
        cv.circle(image_src_copy, (x, y), 5, (0, 255, 0), -1)
        cv.imshow('Imagen Origen', image_src_copy)

# Función para manejar los eventos del mouse en la imagen de destino
def mouse_dst(event, x, y, flags, param):
    global dst_points

    if event == cv.EVENT_LBUTTONDOWN and len(dst_points) < 3:
        # Almacenar el punto seleccionado en la lista de puntos de destino
        dst_points.append([x, y])
        cv.circle(image_dst_copy, (x, y), 5, (0, 0, 255), -1)
        cv.imshow('Imagen Destino', image_dst_copy)

    # Si ya se han seleccionado tres puntos en ambas imágenes, aplicar la transformación afín
    if len(src_points) == 3 and len(dst_points) == 3:
        src_points_np = np.float32(src_points)
        dst_points_np = np.float32(dst_points)

        # Calcular la matriz de transformación afín
        M = cv.getAffineTransform(src_points_np, dst_points_np)

        # Aplicar la transformación afín en la imagen de origen
        h, w = image_dst.shape[:2]
        transformed_image = cv.warpAffine(image_src, M, (w, h))

        cv.imshow('Imagen Transformada', transformed_image)

cv.imshow('Imagen Origen', image_src)
cv.setMouseCallback('Imagen Origen', mouse_src)

cv.imshow('Imagen Destino', image_dst)
cv.setMouseCallback('Imagen Destino', mouse_dst)

cv.waitKey(0)
cv.destroyAllWindows()


#### Calcular la imagen especular a partir de una imagen.

Este código genera una imagen especular (reflejo horizontal) de una imagen original cargada. Utiliza la función cv.flip con un parámetro de 1 para crear el reflejo, volteando la imagen de izquierda a derecha. Luego, muestra tanto la imagen original como su versión reflejada en ventanas separadas, permitiendo al usuario comparar ambas imágenes.

In [6]:
import cv2 as cv

image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Calcular la imagen especular (reflejo horizontal)
mirror_image = cv.flip(image, 1)  # El segundo parámetro 1 indica voltear horizontalmente

cv.imshow('Imagen Original', image)
cv.imshow('Imagen Especular', mirror_image)

cv.waitKey(0)
cv.destroyAllWindows()


#### Tratar una recta que será el eje de reflexión y “reflejar” la imagen. 

- Reflejo Vertical: El primer código divide la imagen en dos mitades verticales (izquierda y derecha). Luego, en la primera versión, la mitad derecha se convierte en un reflejo de la izquierda, y en la segunda, la mitad izquierda refleja la derecha. Esto crea reflejos a lo largo de un eje vertical.

- Reflejo Horizontal: El segundo código divide la imagen en dos mitades horizontales (superior e inferior). Genera una versión en la que la mitad superior es un reflejo de la inferior y otra en la que la mitad inferior refleja la superior. Esto produce reflejos a lo largo de un eje horizontal.

- Reflejo Diagonal: El tercer código crea un reflejo diagonal intercambiando píxeles a lo largo de la diagonal principal de la imagen (convertida en cuadrada si es necesario), generando una simetría sobre esa línea.

Reflexión respecto a una recta vertical

In [17]:
import cv2 as cv

image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

mitad_vertical = image.shape[1] // 2

# Asegurarse de que la imagen tenga un número par de columnas
if image.shape[1] % 2 != 0:
    image = image[:, :-1] 

image2 = image.copy()

# Dividir la imagen en dos partes
image[:, :mitad_vertical] = cv.flip(image[:, mitad_vertical:], 1)
image2[:, mitad_vertical:] = cv.flip(image2[:, :mitad_vertical], 1)

cv.imshow('Imagen reflejada a la derecha', image)
cv.imshow('Imagen reflejada a la izquierda', image2)


cv.waitKey(0)
cv.destroyAllWindows()

Reflexión respecto a una recta horizontal

In [15]:
import cv2 as cv

image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

mitad_horizontal = image.shape[0] // 2

# Asegurarse de que la imagen tenga un número par de filas
if image.shape[0] % 2 != 0:
    image = image[:-1, :] 

image2 = image.copy()

# Dividir la imagen en dos partes
image[:mitad_horizontal, :] = cv.flip(image[mitad_horizontal:, :], 0)
image2[mitad_horizontal:, :] = cv.flip(image2[:mitad_horizontal, :], 0)

cv.imshow('Imagen reflejada hacia arriba', image)
cv.imshow('Imagen reflejada hacia abajo', image2)

cv.waitKey(0)
cv.destroyAllWindows()


Reflexión respecto a la diagonal principal.

In [8]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

rows, cols = image.shape[:2]

# Asegurarse de que la imagen sea cuadrada
if rows != cols:
    size = min(rows, cols)
    image = cv.resize(image, (size, size))

image_reflection = image.copy()

# Realizar el reflejo a lo largo de la diagonal
for i in range(size):
    for j in range(i, size):
        # Intercambia los píxeles a lo largo de la diagonal
        image_reflection[i, j], image_reflection[j, i] = image_reflection[j, i], image_reflection[i, j]

cv.imshow('Imagen original', image)
cv.imshow('Imagen reflejada diagonalmente', image_reflection)

cv.waitKey(0)
cv.destroyAllWindows()
